# Yanling single-cell analysis

In [ ]:
from cellassign import assign_cats
import gzip
import itertools as itl
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import os
import pandas as pd
import scanpy as sc
import scanpy.external as sce
import seaborn as sns
import triku as tk

In [ ]:
from scipy.stats import chi2_contingency

In [ ]:
# Palettes for UMAP gene expression

magma = [plt.get_cmap('magma')(i) for i in np.linspace(0,1, 80)]
magma[0] = (0.88, 0.88, 0.88, 1)
magma = mpl.colors.LinearSegmentedColormap.from_list("", magma[:65])

In [ ]:
mpl.rcParams['figure.dpi'] = 200

In [ ]:
def plot_batch_abundance(adatax, plot_status=False):
    samples, clusters, list_prop = sorted(dict.fromkeys(adatax.obs['batch'])), sorted(dict.fromkeys(adatax.obs['leiden'])), []
    product = itl.product(*[samples, clusters])

    correction_factor = {sample: len(adatax)/(len(samples) * np.sum(adatax.obs['batch'] == sample)) for sample in samples}

    for sample, cluster in product:
        list_prop.append(correction_factor[sample] * 
                         len(adatax[(adatax.obs['leiden'] == cluster) & (adatax.obs['batch'] == sample)])/
                         len(adatax[adatax.obs['leiden'] == cluster]))

    df = pd.DataFrame({'x':clusters * len(samples), 'y':list_prop, 'hue':sorted(samples * len(clusters))})
    if plot_status:
        df['hue_status'] = [i[0] for i in df['hue']]
        fig, axs = plt.subplots(1, 2, figsize=(16,4))
        sns.barplot(x='x', y='y', hue='hue', data=df, ax=axs[0])
        sns.barplot(x='x', y='y', hue='hue_status', data=df, ax=axs[1])
    else:
        fig, axs = plt.subplots(1, 1, figsize=(8,4))
        sns.barplot(x='x', y='y', hue='hue', data=df, ax=axs)

In [ ]:
fastq_dir = f'{os.getcwd()}/fastq' 
mouse_gencode_dir = "/media/seth/SETH_DATA/SETH_Alex/Programs/mouse_GRCm38_gencode.v31"

In [ ]:
prefixes = ['KOD11', 'KOD12', 'WT1', 'WT2']

In [ ]:
seed = 0

In [ ]:
# def adata_plot_KOvsWT(adata, dict_names, do_return=False):
    fig, ax = plt.subplots(2, 1, figsize=(9, 12))
    
    df_proportions_KO_WT = pd.DataFrame(columns=['KO', 'WT'], index=dict_names.keys())
    df_counts_KO_WT = pd.DataFrame(columns=['KO', 'WT'], index=dict_names.keys())
    df_pval = pd.DataFrame(columns=['p-val'], index=dict_names.keys())
    
    counts_KO_all = len(adata_all_harmony[adata_all_harmony.obs['condition'] == 'KO'])
    counts_WT_all = len(adata_all_harmony[adata_all_harmony.obs['condition'] == 'WT'])
    
    print(f'N KO cells: {counts_KO_all}, {counts_KO_all/len(adata_all_harmony)}% | N WT cells: {counts_WT_all}, {counts_WT_all/len(adata_all_harmony)}%')
    
    for cell_type in dict_names.keys():
        adata_sub = adata[adata.obs['cell_type'] == cell_type]
        counts_KO = len(adata_sub[adata_sub.obs['condition'] == 'KO'])
        counts_WT = len(adata_sub[adata_sub.obs['condition'] == 'WT'])

        df_counts_KO_WT.loc[cell_type] = [counts_KO, counts_WT]
        df_proportions_KO_WT.loc[cell_type] = [counts_KO/(counts_KO + counts_WT), counts_WT/(counts_KO + counts_WT)]
        df_pval.loc[cell_type] = chi2_contingency(np.array([[counts_KO, counts_WT], [(counts_KO + counts_WT) * counts_KO_all/len(adata_all_harmony), (counts_KO + counts_WT) * counts_WT_all/len(adata_all_harmony) ]]))[1]
        


    idx_sort = (df_proportions_KO_WT['KO'] - df_proportions_KO_WT['WT']).sort_values(ascending=False).index


    df_proportions_KO_WT.loc[idx_sort].plot(kind='bar', stacked=True, color=['#ff6194', '#3a98ba'], ax=ax[0])
    df_counts_KO_WT.loc[idx_sort].plot(kind='bar', stacked=True, color=['#ff6194', '#3a98ba'], ax=ax[1])
    
    
    for idx, pval in enumerate(df_pval.loc[idx_sort]['p-val'].values):
        if 0.01 < pval < 0.05:
            pval_txt = '*'
        elif 0.001 < pval < 0.01:
            pval_txt = '**'
        elif 0.0001 < pval < 0.001:
            pval_txt = '***'
        elif pval < 0.0001:
            pval_txt = '****'
        else:
            pval_txt = ''
    
        ax[0].text(idx, 1.03, pval_txt, ha='center')
        
        max_val = df_counts_KO_WT.loc[idx_sort].sum(1).max()
        
        ax[1].text(idx, df_counts_KO_WT.loc[idx_sort].sum(1).iloc[idx] + 0.03 * max_val, pval_txt, ha='center')
    
    ax[0].set_ylim([0, 1.1])
    ax[1].set_ylim([0, 1.1 * max_val])
    
    plt.tight_layout()
    
    if do_return:
        return df_proportions_KO_WT, df_counts_KO_WT, df_pval

## Load adatas

In [ ]:
adata_all_harmony = sc.read('adatas/adata_all_harmony.h5')

In [ ]:
sc.pl.umap(adata_all_harmony, color=['batch', 'condition', 'cell_type'], ncols=1)

In [ ]:
adata_all_harmony.obs['cell_type'].cat.categories

# Fibroblast analysis

In [ ]:
adata_fb = adata_all_harmony[adata_all_harmony.obs['cell_type'].isin([
     # 'Fibroblast $Acan^+$',
       'Fibroblast $Chf^+$', 'Fibroblast $Clec3b^+$', 'Fibroblast $Coch^+$',
       'Fibroblast $Col8a1^+$', 'Fibroblast $Cxcl1^+$',
       'Fibroblast $Cxcl12^+$', 'Fibroblast $Il1rl1^+$',
       'Fibroblast $Rab37^+$', 'Fibroblast $Serpine2^+$',
       'Fibroblast $Thbs4^+$',])]

In [ ]:
sc.pp.filter_genes(adata_fb, min_counts=1)

In [ ]:
sc.pp.pca(adata_fb, random_state=seed, n_comps=50)
sce.pp.harmony_integrate(adata_fb, key='batch', max_iter_harmony=50, plot_convergence=True)
sc.pp.neighbors(adata_fb, random_state=seed, n_neighbors=int(len(adata_fb) ** 0.5), metric='cosine', use_rep='X_pca_harmony')
tk.tl.triku(adata_fb)

sc.pp.pca(adata_fb, random_state=seed, n_comps=50)
sce.pp.harmony_integrate(adata_fb, key='batch', max_iter_harmony=50, plot_convergence=True)
sc.pp.neighbors(adata_fb, random_state=seed, n_neighbors=int(len(adata_fb) ** 0.5), metric='cosine', use_rep='X_pca_harmony')

In [ ]:
sc.tl.umap(adata_fb)

In [ ]:
sc.pl.umap(adata_fb, color=['batch', 'condition', 'cell_type'], ncols=2)

We see that some populations (*Coch*$^+$ or *Thbs4*$^+$) are separated into two populations. We are going to separate them by finding the corresponding DEGs.

In [ ]:
dict_fbs = {      
                'Fibroblast $Ltbp2^+$': ['Eln', 'Thbs4', 'Ecrg4', 'Chodl', 'Edn3', 'Wnt5a', 'Cilp', 'Crabp1', 'Igfbp2', 'Ltbp2', 'Ism1', 'a', 'Slit2', 'Agt', 'Sema3a', 'Ccdc3', 'Rerg'],
                'Fibroblast $Coch^+$': ['Coch', 'Tnmd', 'Col16a1', 'Col11a1', 'Emid1', 'Crabp1', 'Mafb', 'Fmod', 'Fbn2', 'Cxcr4', 'Gldn', 'Ogn', 'Kera', 'Wfdc1', 'Hs3st6', 'Pi16', 'Ntn5', 'Trib2'],
                'Fibroblast $Cilp2^+$': ['Fmod', 'Comp', 'Tnmd', 'Cilp2', 'Ecrg4', 'Kera', 'Acan', 'Scx', 'Fibin', 'Chad', 'Egfl6', 'Tspan2'], 
                'Fibroblast $Higda^+$': ['Ldha', 'Higda', 'Mt2', 'Ero1l', 'Prg4', 'Mpp6', 'Rnf126', 'Pfkp', 'nip3', 'Slc7a2', 'Rcan1'],
                'Fibroblast $Cxcl12^+$': ['Cxcl12', 'Htra3', 'C1s1', 'Lol', 'Cygb', 'Ly6a', 'Plac9b', 'Lpl', 'Gpc3', 'Ccl19', 'Itm2a', 'Cpxm1', 'Efemp1', 'H19', 'Zim1'],
                'Fibroblast $Cxcl1^+$': ['Cxcl1', 'Tnfaip6', 'Ccl7', 'Has1', 'Cxcl10', 'Ccl2', 'Cxcl2', 'Gfpt2', 'Il6', 'Gpc3', 'Ccl11', 'Clec2d'],
                'Fibroblast $Clec3b^+$': ['Clec3b', 'Fbn1', 'Pi16', 'Scara5', 'Ugp2', 'Cd248', 'Sema3c', 'Entpd2', 'Adamtsl4', 'Mgst3'],
                'Fibroblast $Col8a1^+$': ['Col8a1', 'Eid1', 'Arap1', 'Gpr153', 'Igfbp2', 'Col12a1', 'Phldb2', 'Thbs2', 'Lrrc15', 'Osr2', 'Ccn5', 'Pth1r', 'C1qtnf3', 'Nnmt3'],
                'Fibroblast $Rab37^+$': ['Rab37', 'Col22a1', 'F13a1', 'Htra4', 'Tspan15', 'Rgcc', 'Hbegf', 'Cd55', 'Capg', 'Itga6', 'Pcsk6', 'Sema3c', 'Htra4', 'Sox5', 'Rgs5', 'Aldh1a3', 'Sema3d', 
                                         'Cmtm8', 'Ackr2'],
                'Fibroblast $Cfh^+$': ['Cfh', 'Alpl', 'Lifr', 'Sp7', 'Spp1', 'Runx2', 'Cp', 'Mmp13', 'Ptprd', 'Hp', 'St3Gal6', 'Ibsp', 'Cadm1', 'Lgmn', 'Limmch1', 'Vdr', 'Dapk2', 'Dlx5', 'Sept4'],
                'Fibroblast $Il1rl1^+$': ['Il1rl1', 'Ptgs2', 'Nr4a2', 'Gxylt2', 'Lum', 'Pdpn', 'Mmp2', 'Olfr1033', 'Arpc3', 'Ptges', 'Plin2', 'Npc2', 'Tnc', 'Sh3Pxd2b', 'B2m', 'Twist2'],
                'Fibroblast $Serpine2^+$': ['Serpine2', 'Shox2', 'Wif1', 'Gm48159', 'Col23a1', 'Cd24a', 'Nav2', 'Tfap2c', 'Tanc1', 'Scube3', 'Cblb', 'Fgfr2', 'Olfm', 'Prdm1', 'Alx3', 'Inhba', 'Dio3', 'Rspo1'],
                  }

In [ ]:
sc.tl.leiden(adata_fb, resolution=4)
assign_cats(adata_fb, dict_cats=dict_fbs, min_score=0.4, quantile_gene_sel=0.7, key_added='cell_type')

In [ ]:
sc.pl.umap(adata_fb, color=['batch', 'condition', 'cell_type', 'Pdpn'], ncols=2, frameon=False, cmap=magma)

In [ ]:
sc.pl.umap(adata_fb, color=['Pdpn'], ncols=2, frameon=False, cmap=magma)

In [ ]:
adata_plot_KOvsWT(adata_fb, dict_fbs)

# Keratinocyte analysis

In [ ]:
adata_krt = adata_all_harmony[adata_all_harmony.obs['cell_type'].isin([
    'Keratinocyte $Krt5^+$', 'Keratinocyte $Krt10^+$', 'Keratinocyte $Lor^+$', 'Keratinocyte $Tbx1^+$', 'Keratinocyte $Krt28^+$', 
    'Keratinocyte $Krt75^+$', 'Keratinocyte $Defb6^+$', 'Keratinocyte $Anln^+$', 'Keratinocyte $Cidea^+$'
])]

In [ ]:
sc.pp.filter_genes(adata_krt, min_counts=1)

In [ ]:
sc.pp.pca(adata_krt, random_state=seed, n_comps=50)
sce.pp.harmony_integrate(adata_krt, key='batch', max_iter_harmony=50, plot_convergence=True)
sc.pp.neighbors(adata_krt, random_state=seed, n_neighbors=int(len(adata_krt) ** 0.5), metric='cosine', use_rep='X_pca_harmony')
tk.tl.triku(adata_krt)

sc.pp.pca(adata_krt, random_state=seed, n_comps=50)
sce.pp.harmony_integrate(adata_krt, key='batch', max_iter_harmony=50, plot_convergence=True)
sc.pp.neighbors(adata_krt, random_state=seed, n_neighbors=int(len(adata_krt) ** 0.5), metric='cosine', use_rep='X_pca_harmony')

In [ ]:
sc.tl.umap(adata_krt, min_dist=1)

In [ ]:
sc.pl.umap(adata_krt, color=['batch', 'condition', 'cell_type'], ncols=2)

**LOCATION OF *INFLAMMATORY* KERATINOCYTES**

In [ ]:
sc.pl.umap(adata_krt, color=['Mmp13', 'Pdpn', 'Slfn2', 'Sprr2h', 'Il1f9'], alpha=0.5, ncols=3, cmap=magma)

We see that some populations (*Anln*$^+$ or *Krt5*$^+$) are separated into two populations. We are going to separate them by finding the corresponding DEGs.

In [ ]:
dict_krts = {      
                'Keratinocyte $Anln^+$': ['Anln', 'Prc1', 'Cdk1', 'Cenpf', 'Dnph1', 'Cdca8', 'Birc5', 'Cdca3', 
                                          'Spc24', 'Nusap1', 'Cenpa', 'Ccnb1', 'Racgap1', 'Cdc20'],
                'Keratinocyte $Fxyd1^+$': ['Sparc', 'Vim', 'Lgals1', 'Emp3', 'Crip1', 'S100a4', 'Col6a1', 'Fxyd1', 
                                           'Fstl1', 'Col3a1', 'Matn2', 'Col6a3', 'Zeb2', 'Col5a1', 'Dcn', 'Pcolce'],
                'Keratinocyte $Myh11^+$': ['Tagln', 'Col4a2', 'Col4a1', 'Acta2', 'Myh11', 'Igfbp2', 'Rbpms', 'Myl9', 
                                           'Gm48529', 'Ero1l', 'Anxa6', 'Ptprz1'],
                'Keratinocyte $Krt8^+$': ['Krt18', 'Krt9', 'Cldn3', 'Cystm1', 'Wfdc18', 'Ceacam1', 'Wfdc2', 'Mapre2', 
                                          'Sec14l1', 'Hk1', 'Kcnn4', 'Tesc', 'Rbm47', 'Tpd52l1', 'Ppp1r1b'],
                'Keratinocyte $Krt5^+$': ['Krt5', 'Igfbp2', 'Ly6a', 'Sparc', 'Cdh13', 'Il1r2', 'Efemp1', 'Smoc2', 'Ccdc3', 
                                          'Igfbp6', 'Antxr1', 'Ppic', 'Cotl1', 'Cav1', 'Thy1'], 
                'Keratinocyte $Ifi202b^+$': ['Krt5', 'S100a10', 'Eif5', 'Serpinb2', 'Ifi202b', 'Cxcl16', 'Fosl1', 'Sqstm1', 
                                             'Tiparp', 'Tinagl1', 'Krt16', 'Ptgs2', 'Epgn', 'Adam8', 'Cnksr1', 'Ccdc71l'], 
                'Keratinocyte $Krt10^+$': ['Krt10', 'Krt1', 'Them5', 'Endou', 'Klk8', 'Gltp', 'Calm5', 'Acsl1', 'Chit1', 'Elovl4', 
                                           'Oas1f', 'Ptgs1', 'Tmprss4', 'Ly6g6e', 'Prxl2b', 'Fam57a'],  # Spninous
                'Keratinocyte $Lor^+$': ['Lor', 'Nccrp1', 'Trex2', 'Lce1a1', 'Lce1b', 'Flg', 'Lce1f', 'Hrnr', 'Crct1', 'Lce1g', 
                                         'Lce1e', 'Kprp', 'Il18', 'Lce1m', 'Lce1i', 'Lce1c', 'Lce1h', 'Lce1j', 'Flg2'],  # Granular
                'Keratinocyte $Tbx1^+$': ['Krt17', 'Slc3a2', 'Cxcl14', 'Fgfr1', 'Thbs1', 'Tbx1', 'Dusp6', 'Slc7a8', 'Itgb1', 'Bgn', 
                                          'Smtn', 'Nedd9', 'Col18a1', 'Flna', 'Slpi', 'Lamb1', 'Col4a1', 'Basp1', 'Agrn', 'Tnc'],
                'Keratinocyte $Shisa2^+$': ['Krt14', 'Gja1', 'Krt6a', 'Pdzrn3', 'Pthlh', 'Shisa2', 'Clasrp', 'Pfn2', 'Mgat4a'],
                'Keratinocyte $Krt28^+$': ['Krt28', 'Krt25', 'Krt27', 'Krt71', 'Krt73', 'Tchh', 'Myh14', 'Msx2', 'Nkd2', 
                                           'Ggt1', 'Prss53', 'Dsc1', 'Ttyh2', 'Crnn', 'Msx2', 'Golga7b'],
                'Keratinocyte $Krt35^+$': ['Krt35', 'Mt4', 'Gm49425', 'Msx1', 'S100a3', 'Krtap7-1', 'Krtap8-1', 
                                           'Ly6g6d', 'Selenbp1', 'Krt81', 'Krtap13-1', 'Rab3ip', 'Krtap22-2'],
                'Keratinocyte $Krt75^+$': ['Krt17', 'Tm4sf1', 'Gjb2', 'Wnt11', 'Slc39a6', 'Krt75', 'Fzd5', 'Clu', 'Trim2', 'Sulf2', 
                                           'Lap3', 'Gm44460', 'Sult2b1', 'Slc39a8', 'Gm12446', 'Cyth1', 'Cdr2'],
                'Keratinocyte $Gpx2^+$': ['Gpx2', 'Ly6g6c', 'Krt6a', 'Cpm', 'Cryab', 'Fads3', 'Pinlyp', 'Pgap2', 'Cd82', 
                                          'Klk7', 'Palld', 'Acot1', 'Ss18l2', 'Phactr1', 'Kcne1', 'Gabrp', 'Klk10', 'Ace2'],
                'Keratinocyte $Defb6^+$': ['Krt79', 'Krt17', 'Cst6', 'Ly6g6c', 'Defb6', 'Defb1', 'Klk7', 'Dpysl3', 
                                           'Gata6', 'Alox12e', 'Nebl', 'Teddm3', 'Rbp1', 'Susd2', 'Pdzk1ip1', 'Ano9'],
                'Keratinocyte $Sprr1b^+$': ['Krt16', 'Sprr1b', 'Sprr1a', 'Asprv1', 'Ehf', 'Sbsn', 'Krt80', 'Cnfn', 'Sprr2h', 'Gm9774', 'Ppif', 'Lpin2', 'Il1f9'],
                'Keratinocyte $Cidea^+$': ['Mgst1', 'Cidea', 'Ldhb', 'Acsbg1', 'Pparg', 'Bex1', 'Krt79', 'Psph', 
                                           'Aadac', 'Glrx5', 'Sdc1', 'Eci2', 'Cbr1', 'Scp2', 'Aco1', 'Alad', 'Soat1', 'Cat', 'Lpl'],
                  }

In [ ]:
sc.tl.leiden(adata_krt, resolution=4)
assign_cats(adata_krt, dict_cats=dict_krts, min_score=0.4, quantile_gene_sel=0.7, key_added='cell_type')

In [ ]:
sc.pl.umap(adata_krt, color=['batch', 'condition', 'cell_type'], ncols=2, frameon=False, cmap=magma)

In [ ]:
adata_plot_KOvsWT(adata_krt, dict_krts)

# Immune analysis

In [ ]:
adata_immune = adata_all_harmony[adata_all_harmony.obs['cell_type'].isin([
     'T cell', 'T cell (ILC/gd)?', 'B cell', 'NK cell', 'Macrophage', 'Monocyte', 'Neutrophil', 'Neutrophil*', 'Dendritic cell', 'Langerhans cell', 'Mast cell'
])]

In [ ]:
sc.pp.filter_genes(adata_immune, min_counts=1)

In [ ]:
sc.pp.pca(adata_immune, random_state=seed, n_comps=50)
sce.pp.harmony_integrate(adata_immune, key='batch', max_iter_harmony=50, plot_convergence=True)
sc.pp.neighbors(adata_immune, random_state=seed, n_neighbors=int(len(adata_immune) ** 0.5), metric='cosine', use_rep='X_pca_harmony')
tk.tl.triku(adata_immune)

sc.pp.pca(adata_immune, random_state=seed, n_comps=50)
sce.pp.harmony_integrate(adata_immune, key='batch', max_iter_harmony=50, plot_convergence=True)
sc.pp.neighbors(adata_immune, random_state=seed, n_neighbors=int(len(adata_immune) ** 0.5), metric='cosine', use_rep='X_pca_harmony')

In [ ]:
sc.tl.umap(adata_immune, min_dist=1)

In [ ]:
sc.pl.umap(adata_immune, color=['batch', 'condition', 'cell_type'], ncols=2)

In [ ]:
dict_immune = {    
                  'Langerhans cell': ['Cd74', 'H2-Aa', 'H2-Eb1', 'Mfge8', 'Pgf', 'Cd207', 'Nr4a3', 'Il1r2', 'Pkib', 'Grk3', 'Ktn1', 'Cldn1', 'Pxdc1', 'Dnah2', 'Timd4', 
                                      'Epcam', 'Ccr7', 'St14', 'Mreg', 'Cd40'],
                  'Langerhans $LSR^+$': ['Cd207', 'H2-DMa', 'Ltc4s', 'H2-DMb1', 'Epcam', 'Napsa', 'Hspa1a', 'Lsr', 'Ptpn6', 'Scimp', 'Ptgs1', 'Cyp4f16', 'Tcea3', 'Cd72', 'Cdh1', 'Spint1',
                                         'Dok3', 'Tmem45b', 'Alox5', 'Itgax', 'Fads1', 'Gna15', 'Scap'],
                  'T cell': ['Tnfrsf18', 'Cxcr6', 'Ifngr1', 'Cd52', 'Rora', 'Il7r', 'Aebp2', 'Icos', 'Dgat1', 'Ccr2', 'Shisa5', 'Grap', 'Cd27', 'Pdcd1', 'Ikzf3', 'Il18r1', 
                             'Gata3', 'Spn', 'Tnfrsf4'],
                  'B cell': ['Vpreb3', 'Chchd10', 'Ighm', 'Cd79a', 'Cd79b', 'Stmn1', 'Igll1', 'Dut', 'Top2a', 'Rasgrp2', 'Myb', 'Ssrp1', 'Gimap6', 'Selenoh', 'Apobec3', 'Smc2',
                             'Cks1b', 'Lgals9', 'Ebf1', 'Csrp2', 'Adgrg1', 'Pou2af1', 'Tcf3', 'Hist1h1a', 'Parp1', 'Lig1', 'Dntt', 'Sox4'],
                  'Plasma cell': ['Ighm', 'Igkc', 'Cd79a', 'Ly6d', 'Cd79b', 'Iglc3', 'Ms4a1', 'Iglc2', 'Tnfrsf13c', 'Iglc1', 'Sdc1', 'Cd74', 'Txndc16', 'Ighd'],
                  'NK cell': ['Ctla2a', 'Rgs2', 'Cd3d', 'Cd3g', 'Fermt2', 'Tcrg-C1', 'Nkg7', 'Ctsw', 'Rab4a', 'Cd7', 'Klrk1', 'Trf', 'Itk', 'Ubald2', 'Il2rb', 'Map1b', 'Klrb1b', 'Cd2', 
                              'Trbc2', 'Abcb1b', 'Rasl11b', 'Parp8', 'Tiam1'],
                  'Macrophage': ['Pf4', 'Grn', 'C1qb', 'Dab2', 'C1qc', 'Apoe', 'C1qa', 'Ms4a7', 'Timp2', 'Cd36', 'Mrc1', 'Fxyd2', 'F13a1', 'Igf1', 'Rgl1', 'Ednrb', 'Hgsnat', 'Tgfbi', 'Rnase4', 
                                 'Sash1', 'Folr2', 'Gas6', 'Stab1', 'Ccl8', 'Fcgrt', 'Ms4a6c', 'P2rx7'],
                  'Dendritic cell': ['Cd209a', 'Cd209d', 'Plbd1', 'Aif1', 'Cd74', 'Clec4e', 'Tgfbi', 'Pdpn', 'Cxcl3', 'Vat1', 'Slc39a14', 'Eps8', 'Tppp3', 
                                     'Procr', 'Tarm1', 'Clec4a3', 'Ccl5'],
                  'DC / Langerhans': ['Cd74', 'Cst3', 'Ccr7', 'Bcl2a1a', 'Il4i1', 'Cacnb3', 'Ccl22', 'Fscn1', 'Pkib', 'Wnk1', 'Cd200', 'Gm13546', 'Serpinb9b', 'Relb', 'Cd40', 'Arl5a'],
                  'Monocyte': ['Wfdc17', 'Csf1r', 'F10', 'Ly6c2', 'Gsr', 'Anxa5', 'Ptgs2', 'Cxcl3', 'Pid1', 'Serpinb2', 'Nos2', 'Csf3', 'Vcan', 'Inhba', 'Ptges', 'Saa3', 
                               'F13a1', 'Met', 'Arg1'],
                  'Neutrophil': ['S100a8', 'S100a9', 'Camp', 'Ltf', 'Chil3', 'Tinagl1', 'Stfa1', 'Ngp', 'Chil3', 'Cd177', 'Adpgk', 'Syne1', 'Cebpe', 'Hsd11b1', 'Mgst1', 
                                 'Tst', 'Ly6c2', 'Pilrb2', 'Fcnb', 'Rab44', 'Nfe2', 'Cpne3', 'Ldhc', 'Hmgn2', 'Adgrg3'],
                  'Neutrophil*': ['Acod1', 'Il1f9', 'Rhov', 'Stfa2l1', 'Ccl3', 'Hcar2', 'Slpi', 'Ccrl2', 'Csta3', 'Il1rn', 'Cxcl3', 'G0s2', 'Stfa2', 'Ptgs2', 'Asprv1', 'Fpr1'],
                  'Mast cell': ['Cma1', 'Tpsb2', 'Mcpt4', 'Fdx1', 'Hs3st1', 'Cd55', 'Tpsab1', 'Hdc', 'Tph1', 'Cpa3', 'Fgfr1', 'Gata2', 'Slco2b1', 'Vwa5a', 'Il1rl1', 
                                'Cyp11a1', 'Adora3', 'Kit', 'Ap1s3', 'Hexb', 'Creb3l1', 'Mrgprb1', 'Rab27b', 'Penk', 'Hs6st2', 'Adamts9', 'Akap12', 'Slc45a3', 'Mrgprb2', 'Gnai1']
                  }

In [ ]:
sc.tl.leiden(adata_immune, resolution=1.5)
assign_cats(adata_immune, dict_cats=dict_immune, min_score=0.4, quantile_gene_sel=0.7, key_added='cell_type')

In [ ]:
sc.pl.umap(adata_immune, color=['batch', 'condition', 'leiden', 'cell_type'] 
           , ncols=2, frameon=False, cmap=magma)

In [ ]:
adata_plot_KOvsWT(adata_immune, dict_immune)